<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/Shapefiles/CensusShapefiles_to_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q pip tensorflow geopandas folium mapclassify google-cloud-storage google-cloud-bigquery[pandas] db-dtypes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 

In [2]:
import folium
import os
import requests
import zipfile
import geopandas as gpd
from tqdm import tqdm

import os
import zipfile
from google.cloud import storage, bigquery
import geopandas as gpd
import pandas as pd


The shapefiles are available at https://www2.census.gov/geo/tiger/TIGER2019/

In [10]:
# Path to your service account JSON file
service_account_json = 'nyu-datasets-77d8cc8e92d6.json'

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_json

In [3]:
from google.colab import auth
auth.authenticate_user()

# US States and US Counties

In [5]:
state_url = 'https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip'
states_gdf = gpd.GeoDataFrame.from_file(state_url)

In [6]:
counties_url = 'https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip'
counties_gdf = gpd.GeoDataFrame.from_file(counties_url)

In [7]:
states = states_gdf['STATEFP'].values

In [8]:
counties = counties_gdf.filter(['STATEFP', 'COUNTYFP']).values

# Download state, county, tract, water data to Google Bucket


In [11]:
def download_file_and_upload_to_gcs(url, local_path, bucket_name, destination_blob_name):
    """
    Downloads a file from a URL and uploads it to Google Cloud Storage.

    Args:
    url (str): URL of the file to download.
    local_path (str): Local path to save the file.
    bucket_name (str): Name of the GCS bucket.
    destination_blob_name (str): Destination blob name in the GCS bucket.
    """
    # Download file
    response = requests.get(url)
    with open(local_path, 'wb') as file:
        file.write(response.content)

    # Upload to Google Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(local_path)

In [ ]:
# Download states
download_file_and_upload_to_gcs(state_url, 'maps/tl_2019_us_state.zip', 'census_shapefiles', 'states/tl_2019_us_state.zip')

In [ ]:
# Download counties
download_file_and_upload_to_gcs(counties_url, 'maps/tl_2019_us_county.zip', 'census_shapefiles', 'counties/tl_2019_us_county.zip')

In [ ]:
# Download tracts
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_tract.zip"
    tract_url = f"https://www2.census.gov/geo/tiger/TIGER2019/TRACT/{filename}"
    download_file_and_upload_to_gcs(tract_url, f'maps/{filename}', 'census_shapefiles', f'tracts/{filename}')


In [ ]:
# Download blockgroups
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_bg.zip"
    tract_url = f"https://www2.census.gov/geo/tiger/TIGER2019/BG/{filename}"
    download_file_and_upload_to_gcs(tract_url, f'maps/{filename}', 'census_shapefiles', f'blockgroups/{filename}')

In [ ]:
# Download blocks
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_tabblock10.zip"
    tract_url = f"https://www2.census.gov/geo/tiger/TIGER2019/TABBLOCK/{filename}"
    download_file_and_upload_to_gcs(tract_url, f'maps/{filename}', 'census_shapefiles', f'blocks/{filename}')


In [ ]:
# Download water areas
for STATEFP, COUNTYFP in tqdm(counties):
    filename = f"tl_2019_{STATEFP+COUNTYFP}_areawater.zip"
    tract_url = f"https://www2.census.gov/geo/tiger/TIGER2019/AREAWATER/{filename}"
    download_file_and_upload_to_gcs(tract_url, f'maps/{filename}', 'census_shapefiles', f'areawater/{filename}')

#  Import shapes to Bigquery

In [13]:
def upload_shapefile_to_bigquery(bucket_name, shapefile_blob_name, bigquery_dataset_id, bigquery_table_id, temp_dir='temp', STATEFP=None, COUNTYFP=None):
    # Initialize Google Cloud clients
    storage_client = storage.Client()
    bigquery_client = bigquery.Client()

    # Create the temporary directory if it does not exist
    os.makedirs(temp_dir, exist_ok=True)

    # Download the shapefile ZIP from the bucket
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(shapefile_blob_name)

    # Create the full path for the zip file, ensuring all subdirectories exist
    zip_path = os.path.join(temp_dir, shapefile_blob_name)
    os.makedirs(os.path.dirname(zip_path), exist_ok=True)

    blob.download_to_filename(zip_path)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Find the .shp file inside the extracted folder
    shp_path = None
    for root, dirs, files in os.walk(temp_dir):
        for filename in files:
            if filename.endswith('.shp'):
                shp_path = os.path.join(root, filename)
                break
        if shp_path:
            break

    if not shp_path:
        raise FileNotFoundError("No .shp file found in the extracted ZIP.")

    # Read the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shp_path)

    if STATEFP: gdf['STATEFP'] = STATEFP
    if COUNTYFP: gdf['COUNTYFP'] = COUNTYFP

    # Convert the geometries to a BigQuery-friendly format (e.g., WKT)
    gdf['geometry'] = gdf['geometry'].apply(lambda x: x.wkt)

    # Convert the GeoDataFrame to a Pandas DataFrame
    df = pd.DataFrame(gdf)

    # Create a BigQuery table and upload the DataFrame
    table_id = f"{bigquery_dataset_id}.{bigquery_table_id}"
    job = bigquery_client.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for the job to complete

    # Clean up: remove temporary files and directories
    for root, dirs, files in os.walk(temp_dir, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))


def convert_wkt_to_geograpphy(tablename):

    bigquery_client = bigquery.Client()
    # Convert the WKT column to a GEOGRAPHY column and drop the original WKT column
    sql = f"""
        CREATE OR REPLACE TABLE `{tablename} AS
        SELECT *, ST_GeogFromText(geometry) AS geography
        FROM `{tablename}`;
        ALTER TABLE `{tablename}
        DROP COLUMN geometry;
    """
    query_job = bigquery_client.query(sql)
    query_job.result()  # Wait for the query to complete

In [16]:
def get_geodataframe_from_bigquery(bucket_name, shapefile_blob_name, temp_dir='temp', STATEFP=None, COUNTYFP=None):
    # Initialize Google Cloud clients
    storage_client = storage.Client()
    bigquery_client = bigquery.Client()

    # Create the temporary directory if it does not exist
    os.makedirs(temp_dir, exist_ok=True)

    # Download the shapefile ZIP from the bucket
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(shapefile_blob_name)

    # Create the full path for the zip file, ensuring all subdirectories exist
    zip_path = os.path.join(temp_dir, shapefile_blob_name)
    os.makedirs(os.path.dirname(zip_path), exist_ok=True)

    blob.download_to_filename(zip_path)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Find the .shp file inside the extracted folder
    shp_path = None
    for root, dirs, files in os.walk(temp_dir):
        for filename in files:
            if filename.endswith('.shp'):
                shp_path = os.path.join(root, filename)
                break
        if shp_path:
            break

    if not shp_path:
        raise FileNotFoundError("No .shp file found in the extracted ZIP.")

    # Read the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shp_path)

    if STATEFP: gdf['STATEFP'] = STATEFP
    if COUNTYFP: gdf['COUNTYFP'] = COUNTYFP

    return gdf

In [17]:

def save_geodfs_to_bucket(geodf_list, bucket_name, destination_blob_name, temp_file='temp.geojson'):
    """
    Combines a list of GeoDataFrames and saves them as one GeoDataFrame in a Google Cloud Storage bucket.

    Args:
    geodf_list (list): List of GeoDataFrames.
    bucket_name (str): Name of the Google Cloud Storage bucket.
    destination_blob_name (str): The destination file name in the bucket.
    temp_file (str, optional): Temporary local file name to store combined GeoDataFrame. Defaults to 'temp.geojson'.
    """

    # Combine GeoDataFrames
    combined_geodf = gpd.GeoDataFrame(pd.concat(geodf_list, ignore_index=True))

    # Save combined GeoDataFrame to a file
    combined_geodf.to_file(temp_file, driver='GeoJSON')

    # Initialize Google Cloud Storage client
    storage_client = storage.Client()

    # Upload the file to Google Cloud Storage
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(temp_file)

    # Optionally, clean up the local temporary file
    os.remove(temp_file)

    print(f"Combined GeoDataFrame uploaded to {bucket_name}/{destination_blob_name}")

# Example usage:
# save_geodfs_to_bucket([gdf1, gdf2, gdf3], 'your-bucket-name', 'combined_data.geojson')


In [ ]:
upload_shapefile_to_bigquery('census_shapefiles', 'states/tl_2019_us_state.zip', 'nyu-datasets.shapefiles', 'us_states')

In [ ]:
upload_shapefile_to_bigquery('census_shapefiles', 'counties/tl_2019_us_county.zip', 'nyu-datasets.shapefiles', 'us_counties')

In [ ]:
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_tract.zip"
    upload_shapefile_to_bigquery('census_shapefiles', f'tracts/{filename}', 'nyu-datasets.shapefiles', 'us_tracts')

convert_wkt_to_geograpphy('nyu-datasets.shapefiles.us_tracts')

In [ ]:
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_bg.zip"
    upload_shapefile_to_bigquery('census_shapefiles', f'blockgroups/{filename}', 'nyu-datasets.shapefiles', 'us_blockgroups')

convert_wkt_to_geograpphy('nyu-datasets.shapefiles.us_blockgroups')

In [ ]:
for STATEFP in tqdm(states):
    filename = f"tl_2019_{STATEFP}_tabblock10.zip"
    upload_shapefile_to_bigquery('census_shapefiles', f'blocks/{filename}', 'nyu-datasets.shapefiles', 'us_blocks')

convert_wkt_to_geograpphy('nyu-datasets.shapefiles.us_blocks')

In [ ]:
# Download water areas
for STATEFP in tqdm(states):
  state_gdfs = []
  for s, COUNTYFP in counties:
    if s!=STATEFP: continue
    filename = f"tl_2019_{STATEFP+COUNTYFP}_areawater.zip"
    gdf = get_geodataframe_from_bigquery('census_shapefiles', f'areawater/{filename}', STATEFP=STATEFP, COUNTYFP=COUNTYFP)
    state_gdfs.append(gdf)
  save_geodfs_to_bucket(state_gdfs, 'census_shapefiles', f"areawater_states/tl_2019_{STATEFP}_areawater.geojson", temp_file='temp.geojson')

  2%|▏         | 1/56 [01:02<57:20, 62.56s/it]

Combined GeoDataFrame uploaded to census_shapefiles/areawater_states/tl_2019_54_areawater.geojson


  4%|▎         | 2/56 [02:17<1:02:46, 69.76s/it]

Combined GeoDataFrame uploaded to census_shapefiles/areawater_states/tl_2019_12_areawater.geojson


  5%|▌         | 3/56 [04:03<1:16:15, 86.34s/it]

...

Combined GeoDataFrame uploaded to census_shapefiles/areawater_states/tl_2019_13_areawater.geojson


 41%|████      | 23/56 [29:49<1:27:10, 158.49s/it]

Combined GeoDataFrame uploaded to census_shapefiles/areawater_states/tl_2019_01_areawater.geojson


 43%|████▎     | 24/56 [30:47<1:08:29, 128.42s/it]

Combined GeoDataFrame uploaded to census_shapefiles/areawater_states/tl_2019_49_areawater.geojson


In [ ]:
    upload_shapefile_to_bigquery('census_shapefiles', f'areawater/{filename}', 'nyu-datasets.shapefiles', 'areawater', STATEFP = STATEFP, COUNTYFP = COUNTYFP)

# Area Water

In [ ]:
def download_and_process_geodata(STATEFP, COUNTYFP, YEAR=2019):
    """
    Downloads and processes geospatial data for a specified state and county.

    Args:
    STATEFP (str): State FIPS code.
    COUNTYFP (str): County FIPS code.
    YEAR (int, optional): Year for the dataset. Defaults to 2019.

    Returns:
    GeoDataFrame: A GeoDataFrame containing the processed geospatial data.
    """



    # Creating directory for maps
    os.makedirs('maps', exist_ok=True)

    # Download the boundaries of tracts
    filename = f"tl_{YEAR}_{STATEFP}_tract.zip"
    tract_url = f"https://www2.census.gov/geo/tiger/TIGER{YEAR}/TRACT/{filename}"
    download_file(tract_url, f'maps/{filename}')

    # Extracting the ZIP file
    with zipfile.ZipFile(f'maps/{filename}', 'r') as zip_ref:
        zip_ref.extractall('maps')

    shapefile = f"maps/{filename[:-4]}.shp"
    geo_df = gpd.read_file(shapefile)
    # Keep only the data for the county of interest
    geo_df = geo_df.query(f"COUNTYFP == '{COUNTYFP}'")

    # Download water areas to avoid including them in maps
    water_filename = f"tl_{YEAR}_{STATEFP+COUNTYFP}_areawater.zip"
    water_url = f"https://www2.census.gov/geo/tiger/TIGER{YEAR}/AREAWATER/{water_filename}"
    download_file(water_url, f'maps/{water_filename}')

    # Extracting the water ZIP file
    with zipfile.ZipFile(f'maps/{water_filename}', 'r') as zip_ref:
        zip_ref.extractall('maps')

    water_shapefile = f"maps/{water_filename[:-4]}.shp"
    water_geodf = gpd.read_file(water_shapefile)

    # Remove water areas from the tracts
    geo_df = gpd.overlay(geo_df, water_geodf, how="difference")

    # Keep only variables of interest
    geo_df = geo_df.filter(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAMELSAD', 'ALAND', 'geometry'])

    return geo_df


In [ ]:

# Example usage:
geo_df = download_and_process_geodata("36", "061") # For New York, Manhattan

# Plot the tract boundaries
geo_df.plot(
    figsize=(10, 10),
    color="white",
    edgecolor="black",
    linewidth=0.5,
    zorder=0,
    facecolor="white",
)